# Import library

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import numpy as np
import scipy
import scipy.stats
from scipy.spatial.distance import cdist
import csv
from sklearn import tree
from sklearn import linear_model, neighbors, ensemble, tree
import pandas as pd

#  Load data

In [5]:
DATA_PATH = "/home/trduong/Data/continuous-policy-learning/train_data.csv"
data = pd.read_csv(DATA_PATH)

In [6]:
data.columns

Index(['Age group', 'BMI', 'Diabetes=0?', 'Diabetes=1?',
       'Congestive Heart Failure and/or Cardiomyopathy=0?',
       'Congestive Heart Failure and/or Cardiomyopathy=1?',
       'Valve Replacement=0?', 'Valve Replacement=1?', 'aspirin=0?',
       'aspirin=1?', 'Acetaminophen=0?', 'Acetaminophen=1?',
       'Acetaminophen hi dose=0?', 'Simvastatin=0?', 'Simvastatin=1?',
       'Simvastatin=0?.1', 'Simvastatin=1?.1', 'Fluvastatin=0?',
       'Lovastatin=0?', 'Lovastatin=1?', 'Pravastatin=0?', 'Pravastatin=1?',
       'Rosuvastatin=0?', 'Rosuvastatin=1?', 'sim_dose'],
      dtype='object')

In [9]:
data.head()

,Age group,BMI,Diabetes=0?,Diabetes=1?,Congestive Heart Failure and/or Cardiomyopathy=0?,Congestive Heart Failure and/or Cardiomyopathy=1?,Valve Replacement=0?,Valve Replacement=1?,aspirin=0?,aspirin=1?,...,Simvastatin=0?.1,Simvastatin=1?.1,Fluvastatin=0?,Lovastatin=0?,Lovastatin=1?,Pravastatin=0?,Pravastatin=1?,Rosuvastatin=0?,Rosuvastatin=1?,sim_dose
0,5.0,38.219255,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,60.490611
1,6.0,25.790660,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,23.094080
2,4.0,32.142232,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45.606561
3,8.0,25.870491,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.008341
4,7.0,18.200561,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.634958


In [11]:
data.describe()

,Age group,BMI,Diabetes=0?,Diabetes=1?,Congestive Heart Failure and/or Cardiomyopathy=0?,Congestive Heart Failure and/or Cardiomyopathy=1?,Valve Replacement=0?,Valve Replacement=1?,aspirin=0?,aspirin=1?,...,Simvastatin=0?.1,Simvastatin=1?.1,Fluvastatin=0?,Lovastatin=0?,Lovastatin=1?,Pravastatin=0?,Pravastatin=1?,Rosuvastatin=0?,Rosuvastatin=1?,sim_dose
count,3880.000000,3880.000000,3880.000000,3880.000000,3880.000000,3880.000000,3880.000000,3880.000000,3880.000000,3880.000000,...,3880.000000,3880.000000,3880.000000,3880.000000,3880.000000,3880.000000,3880.000000,3880.000000,3880.000000,3880.000000
mean,5.885825,22.579245,0.485052,0.111082,0.596392,0.127835,0.520103,0.188660,0.568041,0.175773,...,0.435825,0.046134,0.473196,0.470103,0.007216,0.464948,0.013144,0.473454,0.002320,32.032345
std,1.543761,12.227002,0.499841,0.314275,0.490684,0.333949,0.499660,0.391289,0.495413,0.380676,...,0.495928,0.209802,0.499345,0.499170,0.084654,0.498834,0.113907,0.499359,0.048112,15.699950
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.056049
25%,5.000000,20.544420,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.239512
50%,6.000000,24.993541,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,32.480334
75%,7.000000,29.286103,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,42.806489
max,9.000000,79.679009,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,96.498926


In [12]:
for i in data.columns:
    print(i)

Age group
BMI
Diabetes=0?
Diabetes=1?
Congestive Heart Failure and/or Cardiomyopathy=0?
Congestive Heart Failure and/or Cardiomyopathy=1?
Valve Replacement=0?
Valve Replacement=1?
aspirin=0?
aspirin=1?
Acetaminophen=0?
Acetaminophen=1?
Acetaminophen hi dose=0?
Simvastatin=0?
Simvastatin=1?
Simvastatin=0?.1
Simvastatin=1?.1
Fluvastatin=0?
Lovastatin=0?
Lovastatin=1?
Pravastatin=0?
Pravastatin=1?
Rosuvastatin=0?
Rosuvastatin=1?
sim_dose


In [13]:
treatment = 'sim_dose'
outcome = 'BMI'

col = list(data.columns)
 
cov = col.copy()
cov.remove(treatment)
cov.remove(outcome)

features = col.copy()
features.remove(outcome)

In [16]:
from sklearn.ensemble import GradientBoostingRegressor

In [17]:
model_t = GradientBoostingRegressor()
model_t.fit(data[cov], data[treatment])

model_y = GradientBoostingRegressor()
model_y.fit(data[features], data[outcome])


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [18]:
data.head()

,Age group,BMI,Diabetes=0?,Diabetes=1?,Congestive Heart Failure and/or Cardiomyopathy=0?,Congestive Heart Failure and/or Cardiomyopathy=1?,Valve Replacement=0?,Valve Replacement=1?,aspirin=0?,aspirin=1?,...,Simvastatin=0?.1,Simvastatin=1?.1,Fluvastatin=0?,Lovastatin=0?,Lovastatin=1?,Pravastatin=0?,Pravastatin=1?,Rosuvastatin=0?,Rosuvastatin=1?,sim_dose
0,5.0,38.219255,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,60.490611
1,6.0,25.790660,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,23.094080
2,4.0,32.142232,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45.606561
3,8.0,25.870491,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.008341
4,7.0,18.200561,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.634958


$$v_{DR}=\sum_{i=1}^n[\hat{r}(\tau(x_i), x_i) + \frac{1}{hQ_i}K(\frac{\tau(x_i) - t_i}{h})(y_i - \hat{r}(t_i, x_i))]$$

$\hat{r}(t_i, x_i)$ is the dose-response estimator

K is the kernel function

$\tau(x_i)$ is policy function

In [19]:
def gaussian_kernel(u):
    return np.exp(-0.5 * u**2 )/(np.sqrt(2*np.pi))